In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
rng = np.random.default_rng(1234)        # 재현성

Creating directory /Users/taewangoo/Library/Application Support/bioservices 


In [10]:
import pandas as pd

# 경로와 파일명을 실제 파일 경로로 수정하세요.
metabolite_file = 'Metabolite2KEGG.txt'  # Metabolite<tab>KEGG_C
pathway_file = 'CPD2KEGG.txt'         # Pathway<tab>KEGG_C

# 1. 파일 읽기
met_df = pd.read_csv(metabolite_file, sep='\t', dtype=str)
path_df = pd.read_csv(pathway_file, sep='\t', dtype=str, header=None, names=['Pathway', 'KEGG_C'])

In [12]:
met_df

,short,MetaboAnalyst_matching_result,KEGG_C
0,Ala,L-Alanine,C00041
1,Arg,L-Arginine,C00062
2,Asn,L-Asparagine,C00152
3,Asp,L-Aspartic acid,C00049
4,Cit,Citrulline,C00327
5,Gln,L-Glutamine,C00064
6,Glu,L-Glutamic acid,C00025
7,Gly,Glycine,C00037
8,His,L-Histidine,C00135
9,Ile,L-Isoleucine,C00407


In [14]:
# 3) KEGG_C 기준으로 두 테이블 결합 → Metabolite ↔ Pathway 매핑 테이블 생성
mp = met_df.merge(path_df, on='KEGG_C', how='inner')  # inner join으로 양쪽에 모두 있는 항목만

# 4) 관심 경로에 속한 대사체만 필터링
target_paths = ['map00400', 'map00860']
mp_sub = mp[mp['Pathway'].isin(target_paths)]

# 5) 경로별 대사체 리스트 생성
path2met = (
    mp_sub
    .groupby('Pathway')['short']
    .apply(lambda lst: sorted(lst.unique()))
    .reset_index()
)

print(path2met)


    Pathway            short
0  map00400  [Phe, Trp, Tyr]
1  map00860  [Glu, Gly, Thr]


In [31]:
df.columns[15:]

Index(['Ala', 'Arg', 'Asn', 'Asp', 'Cit', 'Gln', 'Glu', 'Gly', 'His', 'Ile',
       ...
       'SM C18:0', 'SM C18:1', 'SM C20:2', 'SM C22:3', 'SM C24:0', 'SM C24:1',
       'SM C26:0', 'SM C26:1', 'H1', 'Azelaic acid'],
      dtype='object', length=182)

In [40]:
import os
import pandas as pd
import numpy as np
from scipy.special import expit
from sklearn.preprocessing import StandardScaler

# 1. Create output directory
os.makedirs('simulation', exist_ok=True)

# 2. Load data
df = pd.read_csv('./181_metabolite_clinical.csv', index_col=0)
metabolite = df.iloc[:, 14:]  # metabolites
clinical    = df.iloc[:, :14]  # clinical data

# 3. Log transform and standardize metabolites
eps = 1e-6
X_log      = np.log(metabolite.values + eps)
X_scaled   = StandardScaler().fit_transform(X_log)
df_pre     = pd.DataFrame(X_scaled, index=metabolite.index, columns=metabolite.columns)

# 4. Define pathway-to-metabolite mapping
mapping = {
    'map00400': ['Phe', 'Trp', 'Tyr'],
    'map00860': ['Glu', 'Gly', 'Thr']
}

# 5. Extract feature matrices
X1 = df_pre[mapping['map00400']].values
X2 = df_pre[mapping['map00860']].values

# 6. Scenario 1: Linear effects
w_linear     = 1.5
betas_linear = [0.1, 0.2, 0.3, 0.4]
for beta in betas_linear:
    eta = beta * (w_linear * X1.sum(axis=1) + w_linear * X2.sum(axis=1))
    pi  = expit(eta)
    y_col = f'y_linear_beta_{beta}'
    out_df = pd.concat([clinical.reset_index(drop=True),
                        pd.Series(np.random.binomial(1, pi), name=y_col)],
                       axis=1)
    out_df.to_csv(f'simulation/linear_beta_{beta}.csv', index=False)

# 7. Scenario 2: Interaction effects
beta_inter    = 0.4
interaction_ws = [0.1, 0.2, 0.3, 0.4]
for w_int in interaction_ws:
    inter1 = w_int * (X1[:,0]*X1[:,1] + X1[:,0]*X1[:,2] + X1[:,1]*X1[:,2])
    inter2 = w_int * (X2[:,0]*X2[:,1] + X2[:,0]*X2[:,2] + X2[:,1]*X2[:,2])
    term1  = inter1 + X1.sum(axis=1)
    term2  = inter2 + X2.sum(axis=1)
    eta    = beta_inter * (term1 + term2)
    pi     = expit(eta)
    y_col  = f'y_inter_w_{w_int}'
    out_df = pd.concat([clinical.reset_index(drop=True),
                        pd.Series(np.random.binomial(1, pi), name=y_col)],
                       axis=1)
    out_df.to_csv(f'simulation/interaction_w_{w_int}.csv', index=False)

# 8. Scenario 3: Quadratic effects
betas_quad = [0.6, 0.7, 0.8, 0.9]
w_quad     = np.array([(-1)**(i+1) for i in range(X1.shape[1])])
for beta in betas_quad:
    term1 = (w_quad * (X1**2)).sum(axis=1)
    term2 = (w_quad * (X2**2)).sum(axis=1)
    eta   = beta * (term1 + term2)
    pi    = expit(eta)
    y_col = f'y_quad_beta_{beta}'
    out_df = pd.concat([clinical.reset_index(drop=True),
                        pd.Series(np.random.binomial(1, pi), name=y_col)],
                       axis=1)
    out_df.to_csv(f'simulation/quadratic_beta_{beta}.csv', index=False)

print("Simulation files saved to 'simulation/' directory.")


Simulation files saved to 'simulation/' directory.


In [39]:
df_preprocessed

,Ala,Arg,Asn,Asp,Cit,Gln,Glu,Gly,His,Ile,...,y_linear_beta_0.3,y_linear_beta_0.4,y_inter_w_0.1,y_inter_w_0.2,y_inter_w_0.3,y_inter_w_0.4,y_quad_beta_0.6,y_quad_beta_0.7,y_quad_beta_0.8,y_quad_beta_0.9
Name,,,,,,,,,,,,,,,,,,,,,
HCC_1,0.832374,0.299235,0.150481,-0.837425,-1.537061,-0.292043,-0.908283,-1.558198,-0.562916,0.395770,...,0,0,0,0,0,0,1,0,0,0
HCC_2,-0.176098,0.328912,-0.678582,0.543179,-0.366918,0.122829,-0.164713,-1.075047,-0.395844,-0.182402,...,1,0,0,0,1,1,1,1,0,1
HCC_3,0.963349,0.208747,0.967471,-0.436086,-0.588691,-0.014368,0.276173,-0.429672,1.154016,0.236974,...,0,0,1,0,1,1,0,0,1,1
HCC_4,-0.601098,-0.211193,0.393007,0.916165,-0.980306,-1.715770,0.098390,0.930641,-1.243998,-0.299048,...,0,1,0,0,1,0,0,1,1,1
HCC_5,0.898345,0.239157,-2.961729,0.996950,-0.980306,-0.623063,0.039317,-1.242162,-1.203189,-0.545547,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HCC_196,1.152831,-0.909535,-0.252973,-0.180382,-1.190708,-0.997685,0.083832,-0.386683,-0.722577,0.364618,...,1,0,0,0,1,1,1,1,0,1
HCC_197,0.832374,-0.872526,0.981224,0.858850,-0.471762,0.750062,0.458794,0.328639,0.784063,0.748225,...,0,0,1,1,1,1,0,1,0,0
HCC_198,0.558214,-2.352243,1.180915,0.246622,0.820224,-0.802039,1.106485,0.650467,0.072383,-0.416016,...,0,0,1,1,1,1,0,0,0,0


In [19]:
results['sim_outcome'].sum()

np.int64(200)

In [20]:
results

,Name,HCC,AFP,age,sex,disease,etiology,T.stage,TNM.stage,Group,...,SM C20:2,SM C22:3,SM C24:0,SM C24:1,SM C26:0,SM C26:1,H1,Azelaic acid,sim_prob,sim_outcome
0,HCC_1,1,519.10,61,1,3,1,2.0,2.0,Disease3,...,0.609,0.762,13.60,47.0,0.098,0.292,5739,1.10,1.0,1
1,HCC_2,1,297.80,61,1,3,1,2.0,2.0,Disease3,...,0.416,0.059,14.00,38.9,0.047,0.384,5416,1.02,1.0,1
2,HCC_3,1,1.35,45,1,3,1,2.0,2.0,Disease3,...,0.182,4.670,14.10,46.2,0.043,0.381,6120,0.90,1.0,1
3,HCC_4,1,4233.10,61,1,3,1,4.0,4.0,Disease3,...,0.361,0.598,15.20,53.9,0.255,0.686,3876,1.19,1.0,1
4,HCC_5,1,7.50,64,1,3,1,2.0,2.0,Disease3,...,0.263,0.076,16.30,53.3,0.087,0.434,5904,1.22,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,HCC_196,1,6.20,68,2,3,1,2.0,2.0,Disease3,...,0.383,1.040,9.77,38.2,0.055,0.264,9325,3.22,1.0,1
196,HCC_197,1,14.80,57,1,3,1,2.0,2.0,Disease3,...,0.554,1.140,15.00,45.1,0.195,0.451,3990,0.00,1.0,1
197,HCC_198,1,2.70,71,2,3,1,1.0,1.0,Disease3,...,0.306,0.000,11.00,39.8,0.103,0.092,4829,5.31,1.0,1
198,HCC_199,1,89.90,79,2,3,1,1.0,1.0,Disease3,...,0.448,0.009,8.59,34.5,0.040,0.227,5428,3.22,1.0,1
